<a href="https://colab.research.google.com/github/Seitenshi/Board-Exam-Data-Analytics/blob/main/Neural_Network_w_Auto_select_best_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split



In [ ]:
# Load the data from the uploaded CSV file
url = 'https://raw.githubusercontent.com/Seitenshi/Board-Exam-Data-Analytics/main/BED%20Test%20-%20Combined.csv'
df = pd.read_csv(url)

# Data preprocessing
df['Remarks'] = df['Remarks'].replace({'FAILED': 0, 'PASSED': 1})  # Convert string labels to integers
df = df.dropna()  # Drop rows with missing values
df = df.drop(['Subj01', 'Subj02', 'Subj03', 'Gen. Average'], axis=1)  # Drop unnecessary columns

# Split the data into features (X) and target (y)
X = df.drop('Remarks', axis=1).values
y = df['Remarks'].values

# Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=69)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=69)

# Convert data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
X_val = torch.FloatTensor(X_val)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_val = torch.LongTensor(y_val)
y_test = torch.LongTensor(y_test)


In [ ]:
# Define the improved model
class ImprovedModel(nn.Module):
    def __init__(self, input=8, hidden=164, output=2):
        super(ImprovedModel, self).__init__()
        self.fc1 = nn.Linear(input, hidden)
        self.fc2 = nn.Linear(hidden, output)
        self.dropout = nn.Dropout(0.2)  # Add dropout with 20% probability

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout after activation
        x = self.fc2(x)
        return x


In [ ]:
# Instantiate the improved model
improved_model = ImprovedModel()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(improved_model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
# Training loop with validation and early stopping
epochs = 100
train_losses = []
val_losses = []
best_val_loss = float('inf')
best_model_state = None
for epoch in range(epochs):
    improved_model.train()  # Set the model to training mode
    optimizer.zero_grad()
    outputs = improved_model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # Evaluate on validation set
    improved_model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        val_outputs = improved_model(X_val)
        val_loss = criterion(val_outputs, y_val)
        val_losses.append(val_loss.item())
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = improved_model.state_dict()  # Save the best model weights

    if epoch % 10 == 0:
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

    # Early stopping based on validation loss
    if len(val_losses) > 10 and all(val_losses[-1] > val_losses[-i] for i in range(2, 6)):
        print("Early stopping...")
        break



Epoch 1/100, Train Loss: 0.7828, Val Loss: 0.7548
Epoch 11/100, Train Loss: 0.6846, Val Loss: 0.6735
Epoch 21/100, Train Loss: 0.6929, Val Loss: 0.6655
Epoch 31/100, Train Loss: 0.6959, Val Loss: 0.6405
Epoch 41/100, Train Loss: 0.6718, Val Loss: 0.6279
Epoch 51/100, Train Loss: 0.6674, Val Loss: 0.6171
Epoch 61/100, Train Loss: 0.6732, Val Loss: 0.6097
Epoch 71/100, Train Loss: 0.6558, Val Loss: 0.6034
Epoch 81/100, Train Loss: 0.6450, Val Loss: 0.5987
Epoch 91/100, Train Loss: 0.6422, Val Loss: 0.5953


In [ ]:
# Load the best model and evaluate on the test set
improved_model.load_state_dict(best_model_state)
improved_model.eval()
with torch.no_grad():
    test_outputs = improved_model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test) * 100
    print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 60.00%


In [ ]:
# Checking if the model predicts properly
correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = improved_model.forward(data)

    # Semi Predictions
    print(f'{i+1}.) {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')

    # Checking if correct or not
    if y_val.argmax().item() == y_test[i]:
      correct +=1

print(f'No of correct items: {correct}')

1.) tensor([-0.2792, -0.0290]) 	 0 	 1
2.) tensor([-0.8155,  0.3659]) 	 1 	 1
3.) tensor([-0.7810,  0.3203]) 	 1 	 1
4.) tensor([-0.7843,  0.5515]) 	 1 	 1
5.) tensor([-0.1881, -0.1351]) 	 1 	 1
6.) tensor([-0.2595, -0.1350]) 	 1 	 1
7.) tensor([-0.2510,  0.2369]) 	 0 	 1
8.) tensor([-0.4005,  0.0218]) 	 0 	 1
9.) tensor([-0.7497,  0.3626]) 	 1 	 1
10.) tensor([-0.7625,  0.4566]) 	 1 	 1
11.) tensor([-0.6945,  0.1966]) 	 1 	 1
12.) tensor([-0.2949,  0.0517]) 	 1 	 1
13.) tensor([-0.3029,  0.2100]) 	 1 	 1
14.) tensor([-0.5673,  0.1839]) 	 1 	 1
15.) tensor([-0.6601, -0.1145]) 	 1 	 1
16.) tensor([-0.1696, -0.4130]) 	 0 	 0
17.) tensor([-0.3059,  0.0016]) 	 1 	 1
18.) tensor([-0.3532, -0.0391]) 	 0 	 1
19.) tensor([ 0.0234, -0.2800]) 	 1 	 0
20.) tensor([ 0.0524, -0.0437]) 	 1 	 0
21.) tensor([-1.0128,  0.5475]) 	 1 	 1
22.) tensor([-0.1696, -0.4130]) 	 1 	 0
23.) tensor([-0.3894,  0.0899]) 	 0 	 1
24.) tensor([-0.4028,  0.0820]) 	 1 	 1
25.) tensor([-0.4278,  0.2119]) 	 1 	 1
26.) tens